# Workshop 3:  Backpropagation

In [ ]:
import numpy as np
import random

def get_data(path):
    f = open(path, 'r')
    
    lines = f.readlines()
    
    training_images = np.zeros((len(lines), 784))
    training_labels = np.zeros((len(lines), 10))
    index = 0
    for line in lines:
        line = line.strip()
        label = int(line[0])
        training_images[index, :] = np.fromstring(line[2:], dtype=int, sep=',')
        training_labels[index, label - 1] = 1.0
        index += 1
        

    f.close()
    
    return training_images / 255, training_labels

training_images, training_labels = get_data("mnist_test.csv")
sample_image, sample_label = training_images[0], training_labels[0]

## Our Neural Network
Last week, we implemented the forward pass of our neural network. As a refresher, here is a diagram of our network:
<img src="pics/FCN_2.png" width="600">
For learning backpropagation, it is helpful to be familiar with the letters and notation we used last time to represent different parts of the network. Take some time to examine the diagram and recollect if you need.

## Loss
You probably noticed the box with an L inside it. That figure represents the loss, or error, of our network after forward propagation. The loss compares the network's output to the label that we wanted to see, and gives a measurement of how "wrong" the network was. For our network, we will simply take loss to be the distance between the output vector (Z) and the label for our given sample. The formula is below:
<img src="pics/Loss.png" width="600">
Our goal is to tweak the network's weights until this loss is minimized. If we can accomplish this, then our network will have effectively learned how to read hand-written digits. Think about it: suppose we show the network 80 percent of the total MNIST images, and we succesfully minimize the loss for these images. This means that for these samples, the network's output vector is very close to the given label vector. Then, when we show the network new images from the 20 percent that the network hasn't seen, maybe we can be confident that our network's outputs are very close to the actual labels. Thus, we will be able to predict the label for that new sample without being given the label -- just pick the answer closest to the generated output vector.

So how do we minimize the loss? As it turns out, this can be solved with a useful calculus concept -- gradient descent.

## Gradient Descent
We will now start getting into the math of backpropagation. In order to proceed, you will need to understand the multivariable calculus concepts of partial derivatives and gradients.

To start, lets consider the weight matrix V. Suppose we forward pass an image into our network and then generate some loss. We want to tweak the weights in V in just the right way such that the loss becomes smaller. How can we use multivariable calculus to help us?

If we consider the loss as a multivariable function of V, then we could calculate the gradient of the loss with respect to V.
<img src="pics/Gradient.png" width="600">
From calculus, we know that this gradient will point in the direction of greatest increase of the loss -- and the opposite of the gradient will point in the direction of greatest decrease. Then, we can update V as follows:
<img src="pics/Update.png" width="600">
This update rule basically shifts the weights of V by a small amount in the opposite direction of the gradient. The constant coefficient is the learning rate; it should be a small positive number. We step only a small amount because the gradient only tells us the direction of greatest increase -- not how far we should step. Thus, we step a small amount to be safe. This process is gradient descent.

So how can we calculate the gradient of V? We will need to calculate the partial derivative of L with respect to every weight v[i, j]. But for now, let's start by trying to calculate the partial derivative with respect to weight v[1, 1]. Observe the diagram below:

<img src="pics/Backprop_1.png" width="600">
We see that the loss L is a function of z[1], z[2], ... , z[10]. However, weight v[1, 1] only influences z[1]. It does not influence z[2] through z[10]. Thus, we can use chain rule:
<img src="pics/Chain_1.png" width="600">
Then, we can find the partial derivative of L with respect to V[1, 1]:
<img src="pics/Partial_1.png" width="600">
So now that we have succesfully calculated the partial derivative with respect to V[1, 1], how do we compute the same for any V[i, j]? See if you can write down the formula on a sheet of paper.
<img src="pics/Problem_1.png" width="600">
Next, can you calculate the partial derivative with respect to any output bias C[j]? This should be an easier task.
<img src="pics/Problem_2.png" width="600">
Great! So we've calculated the necessary values to update the weight array V and bias array C. However, we still need to calculate the gradients for weight array W and bias array B. These calculations will be slightly different, because W and B further away from the end of the network.

Let's try to calculate the partial derivative with respect to w[1, 1]. Observe the diagram below:
<img src="pics/Backprop_2.png" width="600">
Again, loss L is a function of z[1], z[2], ... , z[10]. But then, each z[i] is a function of h[1]. Finally, h[1] is a function of w[1, 1]. Knowing this, we can apply the chain rule as follows:
<img src="pics/Chain_2.png" width="600">
This may seem like a large computation, but if you recall, we have already calculated dL/dz[k] for all k = 1, ... , 10.
Thus, we have the following:
<img src="pics/Partial_2.png" width="600">
Wow, that looks intimidating. Maybe copy this down but replace components of the equation by certain symbols, whatever makes it easier for you to digest.
Now, calculate the partial derivative with respect to any weight w[i, j].
<img src="pics/Problem_3.png" width="600">
Calculate the partial derivative with respect to any bias b[i].
<img src="pics/Problem_4.png" width="600">
Now that we have the necessary formulas for gradient descent, we are ready to code. We will pick up where we left off last time. The forward pass methods are all complete, but you must now fill in the backpropagation methods.

In [ ]:
# helper method for sigmoid function
def sigmoid(x):
    return np.where(x >= 0, 
                    1 / (1 + np.exp(-x)), 
                    np.exp(x) / (1 + np.exp(x)))




class NeuralNetwork():
    """
    A Fully Connected Neural Network. There are 784 input layer nodes, 12 hidden layer nodes, and 10 output layer
    nodes.
    """
    def __init__(self):
        
        # Arrays to hold node values
        self.N = np.zeros((784, ))
        self.H = np.zeros((12, ))
        self.Z = np.zeros((10, ))
        
        # Arrays to hold weight values (randomly initialized between -1 and 1)
        self.W = 2 * np.random.rand(784, 12) - 1
        self.V = 2 * np.random.rand(12, 10) - 1
        
        # Arrays to hold biases for hidden and output nodes
        self.B = 2 * np.random.rand(12) - 1
        self.C = 2 * np.random.rand(10) - 1
        
        # Arrays to hold the gradients
        self.W_grad = np.zeros((784, 12))
        self.V_grad = np.zeros((12, 10))
        self.B_grad = np.zeros((12, ))
        self.C_grad = np.zeros((10, ))
        

    def fill_input_nodes(self, x):
        """
        Fills input layer with image x.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        """

        self.N = x
        
        
    def fill_hidden_nodes(self):
        """
        Fills the hidden layer nodes.
        """
        
        H = np.dot(self.N, self.W) + self.B
        self.H = np.tanh(H)
        
    def fill_output_nodes(self):
        """
        Fills the output layer nodes.
        """

        Z = np.dot(self.H, self.V) + self.C
        self.Z = sigmoid(Z)

    
    def forward(self, x):
        """
        Given an image vector x, fills every node in the network.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        """

        self.fill_input_nodes(x)
        self.fill_hidden_nodes()
        self.fill_output_nodes()

    def calculate_loss(self, x, label):
        """
        Given an image vector and its corresponding label vector, calculate the loss.
        
        Parameters:
        x: input vector representing image data, one-dimensional vector
        label: input vector representing the label, a one-dimensional vector. Has a 1 in the position 
               corresponding to the correct answer, and 0s everywhere else.
       
        Returns:
        loss: loss of the network given an image, label pair
        """
        
        out = self.forward(x)
        loss = np.sum((self.Z - label) ** 2)
        return loss
    
    
    #### START WORKING ####
    
    
    def calculate_dL_dVij(self, i, j, label):
        """
        Use the formula you derived to calculate dL/dVi,j.
        Place the result in the appropriate spot in self.V_grad.
        
        Parameters:
        i, j: the indices telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_V_grad(self):
        """
        Use the function you just wrote to fill the entire self.V_grad array.
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_dL_dCj(self, j, label):
        """
        Use the formula you derived to calculate dL/dCj.
        Place the result in the appropriate spot in self.C_grad.
        
        Parameters:
        j: the index telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_C_grad(self):
        """
        Use the function you just wrote to fill the entire self.C_grad array.
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_dL_dWij(self, i, j, label):
        """
        Use the formula you derived to calculate dL/dWi,j.
        Place the result in the appropriate spot in self.W_grad.
        
        At this point, it may be helpful to go back into the .calculate_dL_dVij() method and save some values.
        We will need to reuse some values, so it may not make sense to calculate it all over again.
        
        
        Parameters:
        i, j: the indices telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_W_grad(self):
        """
        Use the function you just wrote to fill the entire self.W_grad array.
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_dL_dBj(self, j, label):
        """
        Use the formula you derived to calculate dL/dBj.
        Place the result in the appropriate spot in self.B_grad.
        
        Parameters:
        j: the index telling which weight to calculate the partial derivative with respect to
        label: a label vector indicating the "correct" answer for the network
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def calculate_B_grad(self):
        """
        Use the function you just wrote to fill the entire self.B_grad array.
        """
        
        ### TODO: Write the method
        
        
        ###
        
    def backpropagate(self, label):
        """
        Perform backprop and fill gradient arrays.
        You should not call this method before calling self.forward()
        
        Parameters:
        label: a label vector indicating the "correct" answer for the network
        """
        
        ### TODO: Write the method
        
        
        ###
    
    def update(self, lr=0.01):
        """
        Use the full gradient arrays to update the weights and biases.
        
        Parameters:
        lr: the learning rate. I set it to 0.01 by default, but we may later find other values to be better.
        """
        
        ### TODO: Write the method
        
        
        ###
        
        
        
        
        
    #### STOP WORKING ####
    
    
    ### NOTE
    # Once we calculate the weight and bias gradients, how can we check that they are correct?
    # Thankfully there is a way to be sure if our code is correct. We simply use the formal definition of a
    # partial derivative -- it is simply a rate of change over an extremely small interval.
    # Thus, we can come up with an estimate of our partial. Our calculated partial should match.
    
    # Each of these methods will select a random partial from its respective gradient array and compare it
    # with the estimate. It will print both the calculated value and the estimated value.
    # If the two numbers printed are close to the same, then your gradient calculations are correct.
    # If they are very different, your calculations are wrong.
    # Feel free to use these to test your implementations.
    
    
    # Parameters:
    # x: sample image
    # y: sample label

    def check_V_grad(self, x, y, perturb=0.00001):
        randi = (random.randrange(12), random.randrange(10))
        self.forward(x)
        self.calculate_dL_dVij(randi[0], randi[1], y)
        test1 = self.V_grad[randi[0], randi[1]]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.V[randi[0], randi[1]] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.V[randi[0], randi[1]] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    def check_W_grad(self, x, y, perturb=0.00001):
        randi = (random.randrange(784), random.randrange(12))
        self.forward(x)
        self.calculate_dL_dWij(randi[0], randi[1], y)
        test1 = self.W_grad[randi[0], randi[1]]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.W[randi[0], randi[1]] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.W[randi[0], randi[1]] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    
    def check_C_grad(self, x, y, perturb=0.00001):
        randi = random.randrange(12)
        self.forward(x)
        self.calculate_dL_dCj(randi, y)
        test1 = self.C_grad[randi]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.C[randi] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.C[randi] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
    def check_B_grad(self, x, y, perturb=0.00001):
        randi = random.randrange(10)
        self.forward(x)
        self.calculate_dL_dBj(randi, y)
        test1 = self.B_grad[randi]
        self.clear()
        loss1 = self.calculate_loss(x, y)
        self.clear()
        self.B[randi] += perturb
        loss2 = self.calculate_loss(x, y)
        self.clear()
        self.B[randi] -= perturb
        
        test2 = (loss2 - loss1) / perturb
        
        print(test1)
        print(test2)
        
        
    def clear(self):
        # Arrays to hold node values
        self.N = np.zeros((784, ))
        self.H = np.zeros((12, ))
        self.Z = np.zeros((10, ))
        
        self.W_grad = np.zeros((784, 12))
        self.V_grad = np.zeros((12, 10))
        self.B_grad = np.zeros((12, ))
        self.C_grad = np.zeros((10, ))
        


Test your code below.